<a href="https://colab.research.google.com/github/ShoumikMahbubRidoy/Arduino/blob/master/Titanic_Machine_Learning_from_Disaste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB,MultinomialNB
from time import time
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve
from sklearn.tree import plot_tree, export_text
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
sns.set()
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
titanic = titanic.drop_duplicates()

In [ ]:
titanic=pd.read_csv('train.csv')
titanic.head()

In [ ]:
titanic.columns

In [ ]:
print("Number Of Rows = %s" % titanic.shape[0])
print("Number Of Columns = %s" % titanic.shape[1])

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
numerical_feats = titanic.dtypes[titanic.dtypes != "object"].index
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = titanic.dtypes[titanic.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))

In [ ]:
sns.distplot(titanic['Survived']);
print("Skewness: %f" % titanic['Survived'].skew())
print("Kurtosis: %f" % titanic['Survived'].kurt())

In [ ]:
for col in numerical_feats:
    print('{:15}'.format(col), 
          'Skewness: {:05.2f}'.format(titanic[col].skew()) , 
          '   ' ,
          'Kurtosis: {:06.2f}'.format(titanic[col].kurt())  
         )

In [ ]:
titanic.describe(include='object').T

In [ ]:
titanic.describe(include=['int','float64']).T

In [ ]:
for i in titanic.columns:
    if(titanic[i].dtype!=type(object)):
        print(i,':\n',titanic[i].unique(),'\n')

In [ ]:
for i in titanic.columns:
    if(titanic[i].dtype!=type(object)):
        q1=titanic[i].quantile(0.25)
        q3=titanic[i].quantile(0.75)
        IQR=q3-q1
        ub=q3+(1.5*IQR)
        lb=q1-(1.5*IQR)
        out_ub=titanic[titanic[i]>ub][i]
        out_lb=titanic[titanic[i]<lb][i]
        print("no of outliers in",i,"is",len(out_ub)+len(out_lb))

In [ ]:
plt.figure(figsize=(10,10))


plt.subplot(2,3,1)
plt.title('Fare')
plt.boxplot(titanic['Fare'],patch_artist=True)


plt.subplot(2,3,2)
plt.title('SibSp')
plt.boxplot(titanic['SibSp'],patch_artist=True)

plt.subplot(2,3,3)
plt.title('Parch')
plt.boxplot(titanic['Parch'],patch_artist=True)

In [ ]:
titanic=titanic.drop(['PassengerId'],1)
titanic.head()

In [ ]:
titanic['Pclass'].unique()

In [ ]:
sns.countplot(x=titanic['Pclass'])

In [ ]:
sns.catplot(x='Pclass',y='Survived',data=titanic,kind='bar')

In [ ]:
titanic['Name'].head()

In [ ]:
titanic['Title']=titanic['Name'].str.extract('([A-Za-z]+)\.',)
titanic['Title'].unique()

In [ ]:
sns.countplot(y=titanic['Title'])

In [ ]:
sns.catplot(y='Title',x='Survived',data=titanic,kind='bar')

In [ ]:
titanic=titanic.drop(['Name'],1)
titanic.head()

In [ ]:
sns.countplot(titanic['Sex'])

In [ ]:
sns.catplot(x='Sex',y='Survived',data=titanic,kind='bar')

In [ ]:
sns.catplot(x='Sex',y='Survived',data=titanic,kind='bar',hue='Pclass')

In [ ]:
titanic['Sex']=titanic['Sex'].map({'male':0,'female':1})

In [ ]:
print(titanic.Pclass.unique())
print(titanic.Sex.unique())

In [ ]:
for i in range(0,2):
    for j in range(0,3):
        print(i,j+1)
        temp_dataset=titanic[(titanic['Sex']==i) &  (titanic['Pclass']==j+1)]['Age'].dropna()
        print(temp_dataset)
        #print(str(temp_dataset.median())+"  "+str(i)+"  "+str(j+1))
        titanic.loc[(titanic.Age.isnull()) & (titanic.Sex==i) & (titanic.Pclass==j+1),'Age']=int(temp_dataset.median())

In [ ]:
titanic['Fare']=titanic['Fare'].fillna(titanic.loc[(titanic['Pclass']==3) & (titanic['Sex']=="male") & (titanic['Age']<65) & (titanic['Age']>55)].dropna()['Fare'].mean())

In [ ]:
titanic['Fare_Band']=pd.cut(titanic['Fare'],3)
titanic['Fare_Band'].unique()

In [ ]:
titanic.loc[(titanic['Fare']<=170.776),'Fare']=1
titanic.loc[(titanic['Fare']>170.776) & (titanic['Fare']<=314.553),'Fare']=2
titanic.loc[(titanic['Fare']>314.553) & (titanic['Fare']<=513),'Fare']=3
titanic=titanic.drop(['Fare_Band'],1)

In [ ]:
titanic['Fare'].value_counts()

In [ ]:
titanic['Age_Band']=pd.cut(titanic['Age'],5)
titanic['Age_Band'].unique()
sns.catplot(y="Age_Band",x='Survived',data=titanic,kind='bar')

In [ ]:
sns.catplot(y="Age_Band",x='Survived',data=titanic,kind='bar',hue='Pclass')

In [ ]:
g = sns.FacetGrid(titanic, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
g = sns.FacetGrid(titanic, col='Survived',row='Sex')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
g = sns.FacetGrid(titanic, row='Survived',col='Pclass')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
grid = sns.FacetGrid(titanic, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex')
grid.add_legend()

In [ ]:
titanic=titanic.drop(['Age_Band'],1)

In [ ]:
titanic.loc[(titanic['Age']<=16.136),'Age']=1
titanic.loc[(titanic['Age']>16.136) & (titanic['Age']<=32.102),'Age']=2
titanic.loc[(titanic['Age']>32.102) & (titanic['Age']<=48.068),'Age']=3
titanic.loc[(titanic['Age']>48.068) & (titanic['Age']<=64.034),'Age']=4
titanic.loc[(titanic['Age']>64.034) & (titanic['Age']<=80.),'Age']=5
titanic['Age'].unique()

In [ ]:
titanic['FamilySize']=titanic['SibSp']+titanic['Parch']+1
titanic=titanic.drop(['SibSp','Parch'],1)

In [ ]:
sns.catplot(x='FamilySize',y='Survived',data=titanic,kind='bar')

In [ ]:
sns.catplot(x='FamilySize',y='Survived',data=titanic,kind='bar',hue='Sex')

In [ ]:
sns.catplot(x='FamilySize',y='Survived',data=titanic,kind='bar',hue='Pclass')

In [ ]:
titanic=titanic.drop(['Ticket'],1)
titanic.head()

In [ ]:
titanic['Cabin'].isnull().sum()

In [ ]:
titanic['Cabin'].unique()

In [ ]:
titanic['Cabin']=titanic['Cabin'].fillna('U')
titanic['Cabin']=titanic['Cabin'].astype(str).str[0]

In [ ]:
titanic.Cabin.unique()

In [ ]:
sns.catplot(x='Cabin',y='Survived',data=titanic,kind='bar')

In [ ]:
sns.catplot(x='Cabin',y='Survived',data=titanic,col='Sex',kind='bar')

In [ ]:
sns.catplot(x='Cabin',y='Survived',data=titanic,col='Pclass',kind='bar')

In [ ]:
sns.catplot(x='Cabin',y='Survived',data=titanic,col='Pclass',row='Sex',kind='bar')

In [ ]:
titanic=titanic.drop(['Cabin'],1)
titanic.head()

In [ ]:
print(titanic['Embarked'].unique())
print(titanic['Embarked'].isnull().sum())

In [ ]:
titanic['Embarked']=titanic['Embarked'].fillna(titanic['Embarked'].value_counts().index[0])
print(titanic['Embarked'].unique())

In [ ]:
sns.catplot(x='Embarked',y='Survived',data=titanic,kind='bar')

In [ ]:
titanic.head()

In [ ]:
sns.pairplot(titanic)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(titanic.corr(), annot=True)

In [ ]:
titanic.head()

In [ ]:
titanic=pd.get_dummies(columns=['Pclass','Sex','Embarked','Title','Age','Fare'],data=titanic)
titanic.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
from sklearn.model_selection import train_test_split
X = titanic.drop(['Survived'],axis=1)
y = titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.20,random_state =123)
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_train

In [ ]:
X_test=scaler.transform(X_test)
X_test

In [ ]:
from sklearn.model_selection import train_test_split
X = titanic.drop(['Survived'],axis=1)
y = titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.20,random_state =123)

from sklearn.linear_model import LogisticRegression
LR=LogisticRegression(C=0.01)
LR.fit(X_train,y_train)
print('Logistic Regression Train Score :',LR.score(X_train,y_train))
print('Logistic Regression Test Score :',LR.score(X_test,y_test))

In [ ]:
from sklearn.metrics import confusion_matrix
CM_LR=confusion_matrix(y_test,y_predict_LR)
print(CM_LR)
sns.heatmap(CM_LR,center=True,annot=True)

In [ ]:
y_predict_LR=LR.predict(X_test)
print('Y Predict: ',y_predict_LR)

In [ ]:
from sklearn.metrics import accuracy_score
acc_LR=accuracy_score(y_test,y_predict_LR)
print(acc_LR)

In [ ]:
from sklearn.metrics import classification_report
CR_LR=classification_report(y_test,y_predict_LR)
print(CR_LR)

In [ ]:
from sklearn.metrics import roc_curve,auc
LR_tpr,LR_fpr,threshold=roc_curve(y_test,y_predict_LR)
LR_auc = auc(LR_tpr, LR_fpr)
print('LR_tpr: ',LR_tpr)
print('LR_fpr: ',LR_fpr)
print('threshold: ',threshold)

#Draw ROC Curve && AUC [Area Under The Curve]
plt.figure(figsize=(9, 8))
plt.plot(LR_tpr, LR_fpr, marker='o', label='Logistic Regression (auc = %0.3f)' % LR_auc)
plt.ylabel('True Positive Rate -->')
plt.xlabel('False Positive Rate -->')

plt.legend()

plt.show()

In [ ]:
from sklearn.svm import SVC
SVM=SVC(C=1,kernel='rbf')
SVM.fit(X_train,y_train)
print('Logistic Regression Train Score :',SVM.score(X_train,y_train))
print('Logistic Regression Test Score :',SVM.score(X_test,y_test))

In [ ]:
y_predict_SVM=SVM.predict(X_test)
print(y_predict_SVM)

In [ ]:
from sklearn.metrics import  confusion_matrix
CM_SVM=confusion_matrix(y_test,y_predict_SVM)
print(CM_SVM)
sns.heatmap(CM_SVM,center=True,annot=True)

In [ ]:
from sklearn.metrics import accuracy_score
acc_svm=accuracy_score(y_test,y_predict_SVM)
print(acc_svm)

In [ ]:
from sklearn.metrics import classification_report
CR_SVM=classification_report(y_test,y_predict_SVM)
print(CR_SVM)

In [ ]:
from sklearn.metrics import roc_curve,auc
svm_tpr,svm_fpr,threshold=roc_curve(y_test,y_predict_SVM)
svm_auc=auc(svm_tpr,svm_fpr)
print('svm_tpr',svm_tpr)
print('svm_fpr',svm_fpr)
print('threshold',threshold)


#Draw ROC Curve && AUC [Area Under The Curve]
plt.figure(figsize=(9, 5))
plt.plot(svm_tpr, svm_fpr, linestyle=':', label='SVM (auc = %0.3f)' % svm_auc)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')

plt.legend()

plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=13,metric = 'minkowski', p = 1)
knn.fit(X_train,y_train)
print('KNN Train Score :',knn.score(X_train,y_train))
print('KNN Test Score :',knn.score(X_test,y_test))

In [ ]:
y_predict_knn=knn.predict(X_test)
print(y_predict_knn)

In [ ]:
from sklearn.metrics import confusion_matrix
cm_knn=confusion_matrix(y_test,y_predict_knn)
print(cm_knn)
sns.heatmap(cm_knn,center=True,annot=True)

In [ ]:
from sklearn.metrics import accuracy_score
acc_knn=accuracy_score(y_test,y_predict_knn)
print(acc_knn)

In [ ]:
from sklearn.metrics import classification_report
cr_knn=classification_report(y_test,y_predict_knn)
print(cr_knn)

In [ ]:
from sklearn.metrics import roc_curve,auc
knn_tpr,knn_fpr,threshold=roc_curve(y_test,y_predict_knn)
print('True Positive Rate',knn_tpr)
print('False Positive Rate',knn_fpr)
print('threshold',threshold)

knn_auc=auc(knn_tpr,knn_fpr)

#Draw ROC Curve && AUC [Area Under The Curve]
plt.figure(figsize=(9, 5))
plt.plot(knn_tpr, knn_fpr, linestyle='--', label='KNN (auc = %0.3f)' %knn_auc)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')

plt.legend()

plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(max_depth=7)
dt.fit(X_train,y_train)
print(' DecisionTree Train Score :',dt.score(X_train,y_train))
print(' DecisionTree Test Score :',dt.score(X_test,y_test))
print('DecisionTree Classifier Model feature importances are :\n ' , dt.feature_importances_)

In [ ]:
y_predict_dt=dt.predict(X_test)
print(y_predict_dt)

In [ ]:
from sklearn.metrics import accuracy_score
acc_dt=accuracy_score(y_test,y_predict_dt)
print(acc_dt)

In [ ]:
from sklearn.metrics import confusion_matrix
cm_dt=confusion_matrix(y_test,y_predict_dt)
print(cm_dt)
sns.heatmap(cm_dt,center=True,annot=True)

In [ ]:
from sklearn.metrics import classification_report
cr_dt=classification_report(y_test,y_predict_dt)
print(cr_dt)

In [ ]:
from sklearn.metrics import roc_curve,auc
dt_tpr,dt_fpr,threshold=roc_curve(y_test,y_predict_dt)
print('True Positive Rate',dt_tpr)
print('False Positive Rate',dt_fpr)
print('threshold',threshold)

dt_auc=auc(dt_tpr,dt_fpr)

#Draw ROC Curve && AUC [Area Under The Curve]
plt.figure(figsize=(9, 5))
plt.plot(dt_tpr, dt_fpr, linestyle='--', label='DT (auc = %0.3f)' %dt_auc)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')

plt.legend()

plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
rf=RandomForestClassifier(criterion = 'entropy',max_depth=6,n_estimators=9)
rf.fit(X_train,y_train)
print('Random Force Train Score :',rf.score(X_train,y_train))
print('Random Force Test Score :',rf.score(X_test,y_test))
print('Random Force Classifier Model feature importances are :\n ' , rf.feature_importances_)
print('----------------------------------------------------')

In [ ]:
y_pred_rf=rf.predict(X_test)
print('Y Pred',y_pred_rf)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred_rf)
print(cm)
plt.figure(figsize=(9,5))
sns.heatmap(cm,center=True,annot=True)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
acc_rf=accuracy_score(y_test,y_pred_rf)
print('Accuracy Score',acc_rf)

In [ ]:
from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred_rf)
print(cr)

In [ ]:
from sklearn.metrics import roc_curve,auc
rf_tpr,rf_fpr,threshold=roc_curve(y_test,y_pred_rf)
rf_auc=auc(rf_tpr,rf_fpr)
print('rf_tpr Value  : ', rf_tpr)
print('rf_fpr Value  : ', rf_fpr)
print('thresholds Value  : ', threshold)

#Draw ROC Curve && AUC [Area Under The Curve]

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(rf_tpr, rf_fpr, linestyle='-', label='Random Forest (auc = %0.3f)' % rf_auc)

plt.xlabel('True Positive Rate -->')
plt.ylabel('False Positive Rate -->')

plt.legend()
plt.show()

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
print('XGB Train Score :',xgb.score(X_train,y_train))
print('XGB Test Score :',xgb.score(X_test,y_test))

In [ ]:
y_pred_xgb=xgb.predict(X_test)
print('Y Pred',y_pred_xgb)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred_xgb)
print(cm)
plt.figure(figsize=(9,5))
sns.heatmap(cm,center=True,annot=True)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
acc_xgb=accuracy_score(y_test,y_pred_xgb)
print('Accuracy Score',acc_xgb)

In [ ]:
from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred_xgb)
print(cr)

In [ ]:
from sklearn.metrics import roc_curve,auc
xgb_tpr,xgb_fpr,threshold=roc_curve(y_test,y_pred_xgb)
xgb_auc=auc(xgb_tpr,xgb_fpr)
print('xgb_tpr Value  : ', xgb_tpr)
print('xgb_fpr Value  : ', xgb_fpr)
print('thresholds Value  : ', threshold)

#Draw ROC Curve && AUC [Area Under The Curve]

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(xgb_tpr, xgb_fpr, linestyle='-', label='XGBOOST (auc = %0.3f)' % xgb_auc)

plt.xlabel('True Positive Rate -->')
plt.ylabel('False Positive Rate -->')

plt.legend()
plt.show()

In [ ]:
from sklearn.naive_bayes import BernoulliNB
NB=BernoulliNB()
NB.fit(X_train,y_train)
print('Naive Bayse Train Score',NB.score(X_train,y_train))
print('Naive Bayse Test Score',NB.score(X_test,y_test))

In [ ]:
y_pred_NB = NB.predict(X_test)
y_pred_prob = NB.predict_proba(X_test)
y_pred_prob2=y_pred_prob.astype(int)
print('Y Test' ,y_test)
print('Predicted Value for BernoulliNBModel is : ' , y_pred_NB)
print('Prediction Probabilities Value for BernoulliNBModel is : \n' , y_pred_prob2)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred_NB)
print(cm)
plt.figure(figsize=(9,5))
sns.heatmap(cm,center=True,annot=True)

In [ ]:
from sklearn.metrics import accuracy_score
acc_nb=accuracy_score(y_test,y_pred_NB)
print(acc_nb)

In [ ]:
from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred_NB)
print(cr)

In [ ]:
from sklearn.metrics import roc_curve,auc
nb_tpr,nb_fpr,threshold=roc_curve(y_test,y_pred_NB)
nb_auc=auc(nb_tpr,nb_fpr)
print('nb_tpr Value  : ', nb_tpr)
print('nb_fpr Value  : ', nb_fpr)
print('thresholds Value  : ', threshold)

#Draw ROC Curve && AUC [Area Under The Curve]

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(rf_tpr, rf_fpr, linestyle='-', label='Naive Baias (auc = %0.3f)' % nb_auc)

plt.xlabel('True Positive Rate -->')
plt.ylabel('False Positive Rate -->')

plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

LR_tpr,LR_fpr,threshold=roc_curve(y_test,y_predict_LR)
LR_auc = auc(LR_tpr, LR_fpr)

svm_fpr, svm_tpr, threshold = roc_curve(y_test, y_predict_SVM)
auc_svm = auc(svm_fpr, svm_tpr)

knn_fpr, knn_tpr, threshold = roc_curve(y_test, y_predict_knn)
auc_knn = auc(knn_fpr, knn_tpr)

dt_fpr, dt_tpr, threshold = roc_curve(y_test, y_predict_dt)
auc_dt = auc(dt_fpr, dt_tpr)

rf_fpr, rf_tpr, threshold = roc_curve(y_test, y_pred_rf)
auc_rf = auc(rf_fpr, rf_tpr)


nb_fpr, nb_tpr, threshold = roc_curve(y_test, y_pred_NB)
auc_nb = auc(nb_fpr, nb_tpr)

xgb_fpr, xgb_tpr, threshold = roc_curve(y_test, y_pred_xgb)
auc_xgb = auc(xgb_fpr, xgb_tpr)

plt.figure(figsize=(9, 5))
plt.plot(LR_tpr, LR_fpr, marker='o', label='Logistic Regression (auc = %0.3f)' % LR_auc)
plt.plot(svm_fpr, svm_tpr, linestyle='--', label='SVM (auc = %0.3f)' % auc_svm)
plt.plot(knn_fpr, knn_tpr, linestyle=':', label='KNN (auc = %0.3f)' % auc_knn)
plt.plot(dt_fpr, dt_tpr, linestyle='-', label='DT (auc = %0.3f)' % auc_dt)
plt.plot(rf_fpr, rf_tpr, linestyle='-', label='RF (auc = %0.3f)' % auc_rf)
plt.plot(nb_fpr, nb_tpr, linestyle='--', label='NB (auc = %0.3f)' % auc_nb)
plt.plot(xgb_fpr, xgb_tpr, linestyle='--', label='XGB (auc = %0.3f)' % auc_xgb)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')

plt.legend()

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
n_neighbors = range(1, 31)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
# define grid search
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
model = SVC()
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']
# define grid search
grid = dict(kernel=kernel,C=C,gamma=gamma)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
n_estimators = [10, 100, 1000]
max_features = ['sqrt', 'log2']
# define grid search
grid = dict(n_estimators=n_estimators,max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))